In [1]:
import datetime
import pickle
import numpy as np
import pandas as pd
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import yfinance as yf
from torch.utils.data import Dataset, DataLoader

# --------------------------------------------------------
# 0) SET DEVICE
# --------------------------------------------------------
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# --------------------------------------------------------
# 1) OPTIMIZED TRADING ENVIRONMENT
# --------------------------------------------------------
class TradingEnv(gym.Env):
    def __init__(self, prices, divs, vols, irate, initial_cash=10000, transaction_cost=1e-4):
        super().__init__()
        self.prices_raw = prices.to(device)
        self.divs_raw = divs.to(device)
        self.vols_raw = vols.to(device)
        self.irate_raw = irate.to(device)
        self.initial_cash = initial_cash
        self.transaction_cost = transaction_cost
        self.T, self.N = prices.shape

        # Precompute normalization on GPU
        self.mu_p = prices.mean(0).to(device)
        self.sd_p = prices.std(0).add(1e-6).to(device)
        self.mu_d = divs.mean(0).to(device)
        self.sd_d = divs.std(0).add(1e-6).to(device)
        self.mu_v = vols.mean(0).to(device)
        self.sd_v = vols.std(0).add(1e-6).to(device)
        self.mu_i = irate.mean().to(device)
        self.sd_i = irate.std().add(1e-6).to(device)

        self.action_space = gym.spaces.Box(0.0, 1.0, (self.N+1,), np.float32)
        obs_dim = self.N*3 + 2
        self.observation_space = gym.spaces.Box(-np.inf, np.inf, (obs_dim,), np.float32)

    def reset(self):
        self.current_step = 0
        self.cash = float(self.initial_cash)
        self.holdings = torch.zeros(self.N, device=device)
        return self._get_obs()

    def _get_obs(self):
        t = self.current_step
        p = (self.prices_raw[t] - self.mu_p) / self.sd_p
        d = (self.divs_raw[t] - self.mu_d) / self.sd_d
        v = (self.vols_raw[t] - self.mu_v) / self.sd_v
        i = (self.irate_raw[t] - self.mu_i) / self.sd_i
        obs = torch.cat([p, d, v, 
                        torch.tensor([self.cash], device=device), 
                        torch.tensor([i], device=device)])
        return obs.float().cpu().numpy()

    def step(self, action):
        if self.current_step >= self.T - 1:
            return self._get_obs(), 0.0, True, {}

        a = torch.from_numpy(action).to(device).clamp_min(1e-6)
        a = a / a.sum()

        prices = self.prices_raw[self.current_step].clamp_min(1e-6)
        total_value = (self.holdings * prices).sum() + self.cash

        target_vals = a[:-1] * total_value
        target_cash = a[-1] * total_value
        new_holdings = target_vals / prices

        turnover = (new_holdings - self.holdings).abs() * prices
        cost = self.transaction_cost * turnover.sum()

        next_prices = self.prices_raw[self.current_step+1].clamp_min(1e-6)
        next_total = (new_holdings * next_prices).sum() + target_cash - cost
        reward = (next_total / total_value - 1.0).item()

        self.holdings = new_holdings
        self.cash = (target_cash - cost).item()
        self.current_step += 1
        done = (self.current_step >= self.T - 1)

        return self._get_obs(), reward, done, {}

# --------------------------------------------------------
# 2) DATA LOADING WITH TENSOR CONVERSION
# --------------------------------------------------------
start = datetime.datetime(1990, 1, 1)
end = datetime.datetime(2019, 12, 31)
tickers = [
    "AAPL", "MSFT", "UNH", "LLY", "JPM", "JNJ", "XOM", "WMT", "PG", "CVX",
    "HD", "MRK", "COST", "PEP", "ADBE", "KO", "BAC", "ORCL", "INTC",
    "MCD", "ABT", "CSCO", "QCOM", "DHR", "NKE", "WFC", "TXN", "AMD",
    "NEE", "AMGN", "PM", "HON", "UNP", "UPS", "MS", "LOW", "BA", "IBM",
    "CAT", "MDT", "GS", "GE", "DE", "T", "LRCX", "ADI", "CI", "SYK",
    "MU", "SCHW", "ADP", "MMC", "BDX", "PFE", "ADSK", "SO", "PGR", "TGT",
    "AXP", "AON", "SLB", "CL", "APD", "AEP", "CSX", "F", "GM", "FDX",
    "DG", "NSC", "ITW"]

df = yf.download(tickers, start=start, end=end, auto_adjust=False, actions=True, progress=False)

# Convert to tensors immediately
prices = torch.tensor(df['Close'].ffill().bfill().values.astype(np.float32), device=device)
vols = torch.tensor(df['Volume'].ffill().bfill().values.astype(np.float32), device=device)
divs = torch.tensor(df['Dividends'].fillna(0.0).values.astype(np.float32), device=device)

macro = pdr.DataReader(['DGS10'], 'fred', start, end).reindex(df.index).ffill().bfill()
irate = torch.tensor(macro['DGS10'].values.astype(np.float32), device=device)

env = TradingEnv(prices, divs, vols, irate)

# --------------------------------------------------------
# 3) TENSOR-BASED DATA COLLECTION
# --------------------------------------------------------
buffer = []
def random_policy(state):
    a = np.random.rand(env.N+1).astype(np.float32)
    return a / a.sum()

for _ in range(500):
    s = env.reset()
    done = False
    while not done:
        a = random_policy(s)
        s2, r, done, _ = env.step(a)
        # Store as tensors directly
        s_tensor = torch.from_numpy(s).float()
        a_tensor = torch.from_numpy(a).float()
        s2_tensor = torch.from_numpy(s2).float()
        r_tensor = torch.tensor(r, dtype=torch.float32)
        done_tensor = torch.tensor(done, dtype=torch.bool)
        buffer.append((s_tensor, a_tensor, r_tensor, s2_tensor, done_tensor))
        s = s2

class TorchReplayDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

dataset = TorchReplayDataset(buffer)
replay_loader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=4, pin_memory=True)

# --------------------------------------------------------
# 4) CORRECTED MODEL ARCHITECTURES
# --------------------------------------------------------
class QNetwork(nn.Module):
    def __init__(self, s_dim, a_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(s_dim+a_dim, 256), nn.ReLU(),
            nn.Linear(256, 256), nn.ReLU(),
            nn.Linear(256, 1)
        )
    def forward(self, s, a):
        return self.net(torch.cat([s,a], dim=1))

class VAE(nn.Module):
    def __init__(self, s_dim, a_dim, z_dim=10):
        super().__init__()
        self.z_dim = z_dim
        self.enc = nn.Sequential(
            nn.Linear(s_dim+a_dim, 256), nn.ReLU(),
            nn.Linear(256, 256), nn.ReLU(),
            nn.Linear(256, 2*z_dim)
        )
        self.dec = nn.Sequential(
            nn.Linear(s_dim+z_dim, 256), nn.ReLU(),
            nn.Linear(256, 256), nn.ReLU(),
            nn.Linear(256, a_dim), nn.Sigmoid()  # Corrected activation
        )
    def encode(self, s, a):
        mu_logstd = self.enc(torch.cat([s,a], dim=1))
        mu, logstd = mu_logstd.chunk(2, dim=1)
        return mu, logstd
    def decode(self, s, z=None):
        if z is None:
            z = torch.randn(s.size(0), self.z_dim, device=s.device)
        return self.dec(torch.cat([s,z], dim=1))
    def forward(self, s, a):
        mu, logstd = self.encode(s, a)
        std = torch.exp(logstd.clamp(-4, 15))
        z = mu + std * torch.randn_like(std)
        return self.decode(s, z), mu, std

class Perturbation(nn.Module):
    def __init__(self, s_dim, a_dim, phi=0.05):
        super().__init__()
        self.phi = phi
        self.net = nn.Sequential(
            nn.Linear(s_dim+a_dim, 256), nn.ReLU(),
            nn.Linear(256, a_dim), nn.Tanh()
        )
    def forward(self, s, a):
        return self.phi*self.net(torch.cat([s,a], dim=1))

# --------------------------------------------------------
# 5) OPTIMIZED BCQ AGENT
# --------------------------------------------------------
class BCQAgent:
    def __init__(self, s_dim, a_dim, device):
        self.device = device
        self.action_dim = a_dim
        self.vae = VAE(s_dim, a_dim).to(device)
        self.q1 = QNetwork(s_dim, a_dim).to(device)
        self.q2 = QNetwork(s_dim, a_dim).to(device)
        self.q1_t = QNetwork(s_dim, a_dim).to(device)
        self.q2_t = QNetwork(s_dim, a_dim).to(device)
        self.pert = Perturbation(s_dim, a_dim).to(device)

        self.q1_t.load_state_dict(self.q1.state_dict())
        self.q2_t.load_state_dict(self.q2.state_dict())

        self.opt_vae = optim.Adam(self.vae.parameters(), lr=1e-4)
        self.opt_q1 = optim.Adam(self.q1.parameters(), lr=1e-4)
        self.opt_q2 = optim.Adam(self.q2.parameters(), lr=1e-4)
        self.opt_pert = optim.Adam(self.pert.parameters(), lr=1e-4)

        self.discount = 0.995
        self.tau = 0.005
        self.lmbda = 0.75

    def train_step(self, batch):
        s, a, r, s2, done = [x.to(self.device) for x in batch]
        r = r.unsqueeze(1)
        done = done.unsqueeze(1)

        # VAE Update
        recon, mu, std = self.vae(s, a)
        loss_recon = nn.MSELoss()(recon, a)
        loss_kl = -0.5*(1 + torch.log(std**2) - mu**2 - std**2).sum(1).mean()
        self.opt_vae.zero_grad()
        (loss_recon + 0.5*loss_kl).backward()
        self.opt_vae.step()

        # Critic Update
        with torch.no_grad():
            s2_rep = s2.unsqueeze(1).repeat(1, 5, 1).view(-1, s2.size(1))
            a_dec = self.vae.decode(s2_rep)
            a_pert = self.pert(s2_rep, a_dec)
            a_perturbed = torch.clamp(a_dec + a_pert, 0.0, 1.0)
            a_perturbed = a_perturbed / a_perturbed.sum(dim=1, keepdim=True)
            
            q1_t = self.q1_t(s2_rep, a_perturbed).view(s2.size(0), 5, 1)
            q2_t = self.q2_t(s2_rep, a_perturbed).view(s2.size(0), 5, 1)
            c_star = self.lmbda * q1_t.min(1)[0] + (1 - self.lmbda) * q1_t.max(1)[0]
            target_q = r + (1 - done) * self.discount * c_star

        q1 = self.q1(s, a)
        loss_q1 = nn.MSELoss()(q1, target_q)
        self.opt_q1.zero_grad(); loss_q1.backward(); self.opt_q1.step()

        q2 = self.q2(s, a)
        loss_q2 = nn.MSELoss()(q2, target_q)
        self.opt_q2.zero_grad(); loss_q2.backward(); self.opt_q2.step()

        # Perturbation Update
        a_p = self.vae.decode(s)
        a_p = a_p + self.pert(s, a_p)
        a_p = torch.clamp(a_p, 0.0, 1.0)
        a_p = a_p / a_p.sum(dim=1, keepdim=True)
        loss_pert = -self.q1(s, a_p).mean()
        self.opt_pert.zero_grad(); loss_pert.backward(); self.opt_pert.step()

        # Target Updates
        for param, target_param in zip(self.q1.parameters(), self.q1_t.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)
        for param, target_param in zip(self.q2.parameters(), self.q2_t.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

    def act(self, state, epsilon=0.0):
        if np.random.rand() < epsilon:
            a = np.random.rand(self.action_dim).astype(np.float32)
            return a / (a.sum() + 1e-6)
        
        s = torch.FloatTensor(state).to(self.device).unsqueeze(0)
        with torch.no_grad():
            s_rep = s.repeat(10, 1)
            a_samp = self.vae.decode(s_rep)
            a_pert = self.pert(s_rep, a_samp)
            a_perturbed = torch.clamp(a_samp + a_pert, 0.0, 1.0)
            a_perturbed = a_perturbed / (a_perturbed.sum(dim=1, keepdim=True) + 1e-6)
            q_vals = self.q1(s_rep, a_perturbed)
            best = a_perturbed[q_vals.argmax()].cpu().numpy()
        return best

# --------------------------------------------------------
# 6) TRAINING LOOP WITH FIXED DATA HANDLING
# --------------------------------------------------------
agent = BCQAgent(env.observation_space.shape[0], env.action_space.shape[0], device)
num_epochs = 50
history = []
epsilon_start, epsilon_final = 1.0, 0.01
decay = np.log(epsilon_final/epsilon_start) / (num_epochs * (len(dataset)//256))

for epoch in range(1, num_epochs+1):
    for batch in replay_loader:
        agent.train_step(batch)

    epsilon = epsilon_start * np.exp(decay * epoch * (len(dataset)//256))
    returns = []
    for _ in range(10):
        s, done, ep_ret = env.reset(), False, 0
        while not done:
            a = agent.act(s, epsilon)
            s, r, done, _ = env.step(a)
            ep_ret += r
        returns.append(ep_ret)
    avg_ret = np.mean(returns)
    history.append(avg_ret)
    print(f"Epoch {epoch}  AvgRet {avg_ret:.4f}  ε={epsilon:.4f}")

# --------------------------------------------------------
# 7) PLOTTING
# --------------------------------------------------------
# Training curves
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(range(1,num_epochs+1), history, marker='o')
plt.xlabel('Epoch'); plt.ylabel('Avg Eval Return')
plt.title('BCQ Training Progress'); plt.grid(True)

epsilons = [epsilon_start * np.exp(decay*s) for s in range(global_step)]
plt.subplot(1,2,2)
plt.plot(epsilons)
plt.xlabel('Training Step'); plt.ylabel('Epsilon')
plt.title('ε-Decay Schedule'); plt.grid(True)
plt.tight_layout()
plt.show()

# --------------------------------------------------------
# 8) FINAL PORTFOLIO VS S&P 500
# --------------------------------------------------------
# (Your original code; now env and agent are on GPU, data already loaded.)
sp = yf.download('^GSPC', start=start, end=end, auto_adjust=False)['Close']\
       .ffill().bfill().values.astype(np.float32)
state = env.reset(); done=False
portfolio_values = []
while not done:
    total = (env.holdings * env.prices_raw[env.current_step]).sum().item() + env.cash
    portfolio_values.append(total)
    a = agent.act(state)
    state, _, done, _ = env.step(a)

sp500_values = sp[:len(portfolio_values)]
port_norm = np.array(portfolio_values) / portfolio_values[0]
sp_norm   = sp500_values / sp500_values[0]

plt.figure(figsize=(10,5))
plt.plot(port_norm, label='Agent Portfolio')
plt.plot(sp_norm,   label='S&P 500')
plt.xlabel('Time Step'); plt.ylabel('Normalized Value')
plt.title('Normalized Portfolio vs. S&P 500')
plt.legend(); plt.grid(True); plt.tight_layout()
plt.show()


KeyboardInterrupt: 

In [ ]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

# 1) Download S&P 500 history over the same date range as your env
sp = yf.download('^GSPC', start=start, end=end, auto_adjust=False)['Close'] \
        .ffill().bfill().values.astype(np.float32)

# 2) Roll out your final policy and record only portfolio values
state = env.reset()
done = False
portfolio_values = []

while not done:
    # a) compute current portfolio total value
    total_val = (env.holdings * env.prices_raw[env.current_step]).sum() + env.cash
    portfolio_values.append(total_val)

    # b) step the environment
    action = agent.act(state)
    state, _, done, _ = env.step(action)

# 3) Align S&P 500 series to portfolio length
sp500_values = sp[:len(portfolio_values)]

# 4) Normalize both series so they start at 1.0
port_norm = np.array(portfolio_values) / portfolio_values[0]
sp_norm   = np.array(sp500_values)    / sp500_values[0]

# 5) Plot the comparison
plt.figure(figsize=(10, 5))
plt.plot(port_norm, label='Agent Portfolio')
plt.plot(sp_norm,   label='S&P 500')
plt.xlabel('Time Step')
plt.ylabel('Normalized Value')
plt.title('Normalized Portfolio vs. S&P 500 Performance')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [1]:
import gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import yfinance as yf
from pandas_datareader import data as pdr

def evaluate_agent(agent,
                   tickers,
                   start_date='2020-01-01',
                   end_date=None,
                   initial_cash=10_000,
                   transaction_cost=1e-4,
                   device=None):
    """
    Deterministic evaluation of a trained BCQAgent in the TradingEnv.

    Parameters
    ----------
    agent : BCQAgent
        Your trained agent with .act(state) → action (numpy array).
    tickers : list of str
        List of ticker symbols to download.
    start_date : str or datetime
        Data start (inclusive).
    end_date : str or datetime, optional
        Data end (exclusive).  If None, defaults to today.
    initial_cash : float
        Starting cash balance.
    transaction_cost : float
        Per-dollar transaction cost (e.g. 1e-4).
    device : torch.device, optional
        To which the agent should be transferred (assumes agent is already on device).
    """
    # --------------------------------------------------------------------
    # 1) Download market data
    # --------------------------------------------------------------------
    price_df = yf.download(
        tickers,
        start=start_date,
        end=end_date,
        auto_adjust=False
    )["Close"].ffill().bfill()
    prices = price_df.values.astype(np.float32)
    dates  = price_df.index

    vol_df = yf.download(
        tickers,
        start=start_date,
        end=end_date,
        auto_adjust=False
    )["Volume"].ffill().bfill()
    vols = vol_df.values.astype(np.float32)

    # dividends
    divs_df = pd.DataFrame(index=price_df.index, columns=tickers).fillna(0.0)
    for t in tickers:
        d = yf.Ticker(t).dividends.tz_localize(None)
        divs_df[t] = d.reindex(price_df.index).ffill().fillna(0.0)
    divs = divs_df.values.astype(np.float32)

    # interest rate (10-year Treasury)
    macro = pdr.DataReader(['DGS10'], 'fred',
                           start_date, end_date).reindex(price_df.index).ffill().bfill()
    irate = macro['DGS10'].values.astype(np.float32)

    # S&P 500 benchmark
    sp = yf.download('^GSPC', start=start_date, end=end_date,
                     auto_adjust=False)["Close"].ffill().bfill()
    snp = sp.values.astype(np.float32)

    # --------------------------------------------------------------------
    # 2) Build evaluation environment
    # --------------------------------------------------------------------
    env = TradingEnv(prices, divs, vols, irate,
                     initial_cash=initial_cash,
                     transaction_cost=transaction_cost)
    state = env.reset()
    agent.device = device  # if needed

    # --------------------------------------------------------------------
    # 3) Run one episode, collecting value & holdings history
    # --------------------------------------------------------------------
    portf_vals, bench_vals = [], []
    holdings_history, dates_hist = [], []

    done = False
    while not done:
        action = agent.act(state)          # BCQAgent.act → numpy array
        next_state, _, done, _ = env.step(action)
        state = next_state

        t = env.current_step
        current_val = env.cash + (env.holdings * env.prices_raw[t]).sum()
        portf_vals.append(current_val)
        bench_vals.append(snp[t])
        holdings_history.append(env.holdings.copy())
        dates_hist.append(dates[t])

    # --------------------------------------------------------------------
    # 4) Print summary returns
    # --------------------------------------------------------------------
    cum_return   = (portf_vals[-1] / initial_cash - 1) * 100
    bench_return = (bench_vals[-1] / bench_vals[0] - 1) * 100
    print(f"Evaluation from {dates[0].date()} to {dates[-1].date()}")
    print(f"Portfolio Cumulative Return: {cum_return:.2f}%")
    print(f"S&P 500 Cumulative Return: {float(bench_return):.2f}%")

    # --------------------------------------------------------------------
    # 5) Plot normalized portfolio vs. S&P 500
    # --------------------------------------------------------------------
    idx = pd.to_datetime(dates_hist)
    pf_arr    = np.array(portf_vals, dtype=np.float32)
    bench_arr = np.array(bench_vals, dtype=np.float32)

    plt.figure(figsize=(10,5))
    plt.plot(idx, pf_arr / pf_arr[0],    label='Portfolio')
    plt.plot(idx, bench_arr / bench_arr[0], label='S&P 500')
    plt.xlabel('Date'); plt.ylabel('Relative Value (start=1.0)')
    plt.title('Eval: Portfolio vs. S&P 500')
    plt.legend(); plt.tight_layout(); plt.show()

    # --------------------------------------------------------------------
    # 6) Plot individual holdings (8 panels)
    # --------------------------------------------------------------------
    holdings_df = pd.DataFrame(
        np.vstack(holdings_history),
        index=idx,
        columns=tickers
    )
    num_panels        = 8
    tickers_per_panel = int(np.ceil(len(tickers) / num_panels))

    for i in range(num_panels):
        subset = tickers[i*tickers_per_panel:(i+1)*tickers_per_panel]
        if not subset:
            continue
        fig, ax = plt.subplots(figsize=(12,6))
        holdings_df[subset].plot(ax=ax)
        ax.set(
            xlabel='Date',
            ylabel='Shares Held',
            title=f'Holdings (Group {i+1})'
        )
        ax.xaxis.set_major_locator(mdates.AutoDateLocator())
        ax.xaxis.set_major_formatter(
            mdates.ConciseDateFormatter(ax.xaxis.get_major_locator())
        )
        ax.legend(loc='upper left', ncol=2, fontsize='small')
        ax.grid(True)
        plt.tight_layout(); plt.show()


In [ ]:
evaluate_agent(
    agent=agent,
    tickers=fewer_sp500_tickers,
    start_date='2020-01-01',
    end_date='2025-12-31',
    initial_cash=10_000,
    transaction_cost=1e-4,
    device=device
)